## Affine transformation

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import io
from skimage.color import rgb2gray
from scipy import signal
import math
from math import floor, ceil
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

#### Getting the pixel locations  for the image 1

In [ ]:
im1_points = []
def click_and_get_image_points(event, x, y, flags, param):
    if (event == cv2.EVENT_LBUTTONDOWN):
        im1_points.append([x,y])

image1 = cv2.imread("rdj.png", 0)
cv2.namedWindow("rdj.png")
cv2.setMouseCallback("rdj.png", click_and_get_image_points)
while True:
    # display the image and wait for a keypress
    cv2.imshow("rdj.png", image1)
    key = cv2.waitKey(1) & 0xFF
    # if the 'r' key is pressed, reset the cropping region
    if key == ord("r"):
        image = clone.copy()
    # if the 'c' key is pressed, break from the loop
    elif key == ord("c"):
        break

In [ ]:
im1_points = np.array([[389, 493], [570, 514], [471, 646], [365, 711], [532, 725]])
print(im1_points)

#### Getting the pixel locations  for the image 2

In [ ]:

im2_points = []
def click_and_get_image_points2(event, x, y, flags, param):
    if (event == cv2.EVENT_LBUTTONDOWN):
        im2_points.append([x,y])

image2 = cv2.imread("transformedDowney.png", 0)
cv2.namedWindow("transformedDowney.png")
cv2.setMouseCallback("transformedDowney.png", click_and_get_image_points2)
while True:
    # display the image and wait for a keypress
    cv2.imshow("transformedDowney", image2)
    key = cv2.waitKey(1) & 0xFF
    # if the 'r' key is pressed, reset the cropping region
    if key == ord("r"):
        image = clone.copy()
    # if the 'c' key is pressed, break from the loop
    elif key == ord("c"):
        break
        
        

In [ ]:
im2_points

#### Linear system equation

In [18]:
%matplotlib qt
imageDowney = 'rdj.png'
imgDowney = io.imread(imageDowney)
imgDowney = rgb2gray(imgDowney)
# Ensuring that the image is float
imgDowney = ((imgDowney - np.min(imgDowney)) * (1/(np.max(imgDowney) - np.min(imgDowney)) * 1.0)).astype('float')
fig = plt.figure(figsize=(7,7))
img = mpimg.imread('rdj.png')
imgplot = plt.imshow(img)

im1_points = []

def onclick(event):
    im1_points.append([event.xdata, event.ydata])

cid = fig.canvas.mpl_connect('button_press_event', onclick)

In [19]:
theta = math.pi/20.0
rotation = np.array([[math.cos(theta), -math.sin(theta), 0], [math.sin(theta), math.cos(theta), 0], [0,0,1]])
translation = np.array([[1, 0, -5], [0, 1, -10], [0, 0, 1]], np.float)
scaling = np.array([[1.1,0,0], [0, 1.1,0], [0,0,1]], np.float)

T = rotation.dot(scaling.dot(translation))
Tinv = np.linalg.inv(T)


print(T)
print(Tinv)
print(im1_points)

[[  1.08645717  -0.17207791  -3.71150676]
 [  0.17207791   1.08645717 -11.7249613 ]
 [  0.           0.           1.        ]]
[[ 0.89789849  0.14221315  5.        ]
 [-0.14221315  0.89789849 10.        ]
 [ 0.          0.          1.        ]]
[[390.61379581666785, 488.59579699691824], [573.1592026752346, 510.62713920398664], [475.5918300439317, 645.9625270474068], [365.43511900858965, 705.7618844665924], [532.2438528621076, 721.4985574716412]]


In [20]:
[rows, cols] = imgDowney.shape

transformedDowney = np.zeros((rows,cols))

for i in range(0, rows):
    for j in range(0, cols):
        
        currentPoint = np.array([i,j,1])
        transformedPoint = T.dot(currentPoint)
        newi = int(round(transformedPoint[0]))
        newj = int(round(transformedPoint[1]))
        
        if (newi < 0 or newi >= rows or newj < 0 or newj >= cols):
            continue
            
        transformedDowney[i,j] = imgDowney[newi, newj]
    
fig = plt.figure(figsize=(7,7))
imgplot = plt.imshow(transformedDowney, cmap='gray')

im2_points = []

def onclick(event):
    im2_points.append([event.xdata, event.ydata])

cid = fig.canvas.mpl_connect('button_press_event', onclick)

In [21]:
for i in range(len(im1_points)):
    im1_points[i][0] = int(im1_points[i][0])
    im1_points[i][1] = int(im1_points[i][1])
for j in range(len(im2_points)):
    im2_points[j][0] = int(im2_points[j][0]) 
    im2_points[j][1] = int(im2_points[j][1])
    
print(im1_points)
print(im2_points)

[[390, 488], [573, 510], [475, 645], [365, 705], [532, 721]]
[[289, 501], [447, 542], [337, 645], [239, 693], [381, 727]]


In [22]:
q = []
for point in im2_points:
    q.append([point[0]])
    q.append([point[1]])
    
q = np.array(q)

M = []
for point in im1_points:
    M.append([point[0], point[1],1,0,0,0])
    M.append([0,0,0, point[0], point[1],1])
    
M = np.array(M)

#### Solving the linear system equation

In [23]:
p = np.linalg.lstsq(M, q)[0]

C:\Users\siddh\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  """Entry point for launching an IPython kernel.


In [24]:
q

array([[289],
       [501],
       [447],
       [542],
       [337],
       [645],
       [239],
       [693],
       [381],
       [727]])

In [25]:
M

array([[390, 488,   1,   0,   0,   0],
       [  0,   0,   0, 390, 488,   1],
       [573, 510,   1,   0,   0,   0],
       [  0,   0,   0, 573, 510,   1],
       [475, 645,   1,   0,   0,   0],
       [  0,   0,   0, 475, 645,   1],
       [365, 705,   1,   0,   0,   0],
       [  0,   0,   0, 365, 705,   1],
       [532, 721,   1,   0,   0,   0],
       [  0,   0,   0, 532, 721,   1]])

#### Computing the transform matrix and transform inverse for backward transformation

In [26]:
T = p.reshape((2,3))
T = np.append(T, [[0,0,1]], axis=0)

Tinv = np.linalg.inv(T)

print('T: ', T)
print('Tinv: ', Tinv)

T:  [[ 0.87059563 -0.14012478 18.04042776]
 [ 0.1144059   0.89407326 19.39027776]
 [ 0.          0.          1.        ]]
Tinv:  [[  1.12545941   0.17638907 -23.72400228]
 [ -0.14401415   1.09590578 -18.6518406 ]
 [  0.           0.           1.        ]]


#### Computing the backward transformation with bilinear interpolation

In [29]:
rows, cols = imgDowney.shape

top_left = Tinv.dot(np.array([0.0, 0.0, 1.0], float))
top_left = top_left / top_left[2]

bottom_left = Tinv.dot(np.array([0.0, cols-1.0, 1.0], float))
bottom_left = bottom_left / bottom_left[2]

top_right = Tinv.dot(np.array([rows-1.0, 0.0, 1.0], float))
top_right = top_right / top_right[2]

bottom_right = Tinv.dot(np.array([rows-1.0, cols-1.0, 1.0], float))
bottom_right = bottom_right / bottom_right[2]

min_i = np.min([top_left[0], bottom_left[0], top_right[0], bottom_right[0]])
max_i = np.max([top_left[0], bottom_left[0], top_right[0], bottom_right[0]])

min_j = np.min([top_left[1], bottom_left[1], top_right[1], bottom_right[1]])
max_j = np.max([top_left[1], bottom_left[1], top_right[1], bottom_right[1]])


x_sampling = np.linspace(math.floor(min_i), math.ceil(max_i), math.ceil(max_i) - math.floor(min_i)+1)
y_sampling = np.linspace(math.floor(min_j), math.ceil(max_j), math.ceil(max_j) - math.floor(min_j)+1)

out_im2 = np.zeros((len(x_sampling), len(y_sampling)))

for i in range(0, len(x_sampling)):
    for j in range(0, len(y_sampling)):
        
        cur_pt = np.array([x_sampling[i],y_sampling[j],1.0], float)
        transformed_pt = T.dot(cur_pt)
        
        transformed_pt = transformed_pt / transformed_pt[2]
        
        # bilinear interpolation
        x = transformed_pt[0]
        y = transformed_pt[1]
        
        x_rounded_up = int(math.ceil(x))
        x_rounded_down = int(math.floor(x))
        y_rounded_up = int(math.ceil(y))
        y_rounded_down = int(math.floor(y))
        
        ratio_x = x - x_rounded_down
        ratio_y = y - y_rounded_down
        
        try:
            interpolate_x1 = imgDowney[x_rounded_down][y_rounded_down]*(1-ratio_x) + imgDowney[x_rounded_up][y_rounded_down]*ratio_x
            interpolate_x2 = imgDowney[x_rounded_down][y_rounded_up]*(1-ratio_x) + imgDowney[x_rounded_up][y_rounded_up]*ratio_x

            interpolate_y = interpolate_x1*(1-ratio_y) + interpolate_x2*(ratio_y)

            out_im2[i,j] = interpolate_y
        except:
            pass
        
        #new_i = int(round(transformed_pt[0]))
        #new_j = int(round(transformed_pt[1]))
        
        #if (new_i < 0 or new_i >= rows or new_j < 0 or new_j >= cols):
        #    continue
            
        #out_im2[i,j] = image1[new_i, new_j]

In [ ]:
image1_.shape

In [ ]:
# Show the image
fig1 = plt.figure(figsize = (7,7))
plt.title('Affine transformed Image')
plt.imshow(out_im2, cmap='gray')

In [ ]:
cv2.destroyAllWindows()